In [1]:
import os
import PyPDF2
import pytesseract
from pdf2image import convert_from_path
from docx import Document


knowledge_base_path = 'knowledge_base.txt'

with open(knowledge_base_path, 'r', encoding='utf-8') as file:
    knowledge_base = file.read()

print("Knowledge Base Loaded Successfully!")


Knowledge Base Loaded Successfully!


In [2]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')

corpus = knowledge_base.split("\n")

embeddings = model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

print(f"Embeddings generated successfully! Embeddings shape: {embeddings.shape}")

embedding_file_path = "knowledge_base_embeddings.pt"  
torch.save(embeddings, embedding_file_path)

print(f"Embeddings saved successfully to {embedding_file_path}")


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embeddings generated successfully! Embeddings shape: torch.Size([52, 384])
Embeddings saved successfully to knowledge_base_embeddings.pt


In [3]:
import torch

embeddings = torch.load('knowledge_base_embeddings.pt')
print(f"Loaded embeddings shape: {embeddings.shape}")


Loaded embeddings shape: torch.Size([52, 384])


In [7]:
import torch
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

embeddings = torch.load("knowledge_base_embeddings.pt")

with open("knowledge_base.txt", "r") as file:
    corpus = file.readlines()

embeddings_np = embeddings.cpu().numpy()

index = faiss.IndexFlatL2(embeddings_np.shape[1]) 
index.add(embeddings_np)  


In [9]:
import torch
import faiss
from sentence_transformers import SentenceTransformer
from transformers import BartForConditionalGeneration, BartTokenizer

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)


def get_relevant_documents(query, top_k=5):

    query_embedding = sentence_model.encode([query], convert_to_tensor=True)

    query_embedding_np = query_embedding.cpu().numpy()  
    D, I = index.search(query_embedding_np, top_k)  
    relevant_documents = [corpus[i] for i in I[0]]
    return relevant_documents

def generate_answer(query, relevant_docs):
    input_text = " ".join(relevant_docs) + " " + query
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="longest")

    summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=50, max_length=500)
    answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return answer



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
query = input("Ask a legal question: ") 

if query:
    relevant_documents = get_relevant_documents(query, top_k=5)
    answer = generate_answer(query, relevant_documents)

    print(f"Answer: {answer}")

Ask a legal question: what is right to information act
Answer: The Right to Information Act, 2005 was enacted by Parliament in the Fifty-sixth Year of the Republic of India as follows. The Act provides for setting out the practical regime of right to information for citizens to secure access to information under the control of public authorities. It is intended to promote transparency and accountability.
